<a href="https://colab.research.google.com/github/ShreyaC15/BatchNormalization/blob/main/MLP_BatchNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IST597:- Multi-Layer Perceptron - Batch Norm


## Load the libraries

In [ ]:
import os
import numpy as np
import time
import tensorflow as tf
import matplotlib.pyplot as plt
seeds=2785                         # Update seeds
np.random.seed(seeds)
tf.random.set_seed(seeds)

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load data and perform pre-processing

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
X_train = tf.cast(tf.reshape(X_train, (-1, X_train.shape[1]*X_train.shape[2])), dtype=tf.float32)
X_test = tf.cast(tf.reshape(X_test, (-1, X_test.shape[1]*X_test.shape[2])), dtype=tf.float32)

In [ ]:
#X_train = X_train/255.0
#X_test = X_test/255.0

In [ ]:
x_val = X_train[-10000:]
y_val = y_train[-10000:]

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_val=tf.keras.utils.to_categorical(y_val)

In [ ]:
# Split dataset into batches
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(100)
validate_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(100)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(100)

In [ ]:
label_size = y_train.shape[1]

## Build MLP using Eager Execution

In [ ]:
# Define class to build mlp model
# Pre Activation BN
class MLP(object):
  def __init__(self, size_input, size_hidden1,size_hidden2, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden1, self.size_hidden2, self.size_output, self.device =\
    size_input, size_hidden1,size_hidden2, size_output, device
    self.batch=100
    # Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1],stddev=0.1))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden1]))

    # Initialize weights between hidden 1 and hidden layer 2
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2],stddev=0.1))
    # Initialize biases for hidden layer 2
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden2]))

    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_output],stddev=0.1))
    # Initialize biases for output layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))
    
    self.gamma1 = tf.Variable(tf.random.normal([self.batch, self.size_hidden1],stddev=0.1))
    self.gamma2 = tf.Variable(tf.random.normal([self.batch, self.size_hidden2],stddev=0.1))
    self.beta1 = tf.Variable(tf.random.normal([self.batch, self.size_hidden1],stddev=0.1))
    self.beta2 = tf.Variable(tf.random.normal([self.batch, self.size_hidden2],stddev=0.1))

    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3,self.gamma1,self.gamma2,self.beta1,self.beta2]    

  def normalisation(self,X,gamma,beta):
    eps=1e-6
 
    mean= tf.reduce_mean(tf.cast(X, "float"))
    stddev=(tf.reduce_mean(tf.cast((X-mean)**2, "float"))+eps)**(1/2)
    BN= gamma*((X-mean)/stddev)+beta
    return BN
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
    return self.y
  
  def loss(self, y_pred, y_true):
    """
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    """

    #y_pred is the softmax value
    y_true_tf = tf.cast(y_true, dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.keras.losses.CategoricalCrossentropy()(y_true_tf, y_pred_tf)

  def backward(self, X_train, y_train):
    """
    Backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))

  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    # Compute values in hidden layer
    what1 = tf.matmul(X_tf, self.W1) + self.b1  
    hhat1 = tf.nn.relu(self.normalisation(what1,self.gamma1,self.beta1))
    what2 = tf.matmul(hhat1, self.W2) + self.b2   
    hhat2 = tf.nn.relu(self.normalisation(what2,self.gamma2,self.beta2))
    # Compute output
    output = tf.matmul(hhat2, self.W3) + self.b3
    return tf.nn.softmax(output)

  def var(self,y_pred):
    """
    Calculate variance 
    """
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    std_dev = np.std(y_pred_tf) #Calculates standard deviation
    variance = (std_dev**2) # calculate variance
    return variance


In [ ]:
# Define class to build mlp model
# Post Activation BN
class MLP(object):
  def __init__(self, size_input, size_hidden1,size_hidden2, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden: int, size of hidden layer
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden1, self.size_hidden2, self.size_output, self.device =\
    size_input, size_hidden1,size_hidden2, size_output, device
    self.batch=100
    # Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden1],stddev=0.1))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden1]))

    # Initialize weights between hidden 1 and hidden layer 2
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden1, self.size_hidden2],stddev=0.1))
    # Initialize biases for hidden layer 2
    self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden2]))

    self.W3 = tf.Variable(tf.random.normal([self.size_hidden2, self.size_output],stddev=0.1))
    # Initialize biases for output layer
    self.b3 = tf.Variable(tf.random.normal([1, self.size_output]))
    
    self.gamma1 = tf.Variable(tf.random.normal([self.batch, self.size_hidden1],stddev=0.1))
    self.gamma2 = tf.Variable(tf.random.normal([self.batch, self.size_hidden2],stddev=0.1))
    self.beta1 = tf.Variable(tf.random.normal([self.batch, self.size_hidden1],stddev=0.1))
    self.beta2 = tf.Variable(tf.random.normal([self.batch, self.size_hidden2],stddev=0.1))

    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3,self.gamma1,self.gamma2,self.beta1,self.beta2]    

  def normalisation(self,X,gamma,beta):
    eps=1e-6
 
    mean= tf.reduce_mean(tf.cast(X, "float"))
    stddev=(tf.reduce_mean(tf.cast((X-mean)**2, "float"))+eps)**(1/2)
    BN= gamma*((X-mean)/stddev)+beta
    return BN

  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
    return self.y
  
  def loss(self, y_pred, y_true):
    """
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    """

    #y_pred is the softmax value
    y_true_tf = tf.cast(y_true, dtype=tf.float32)
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    return tf.keras.losses.CategoricalCrossentropy()(y_true_tf, y_pred_tf)

  def backward(self, X_train, y_train):
    """
    Backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))

  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    # Compute values in hidden layer
    what1 = tf.matmul(X_tf, self.W1) + self.b1  
    hhat1 = self.normalisation(tf.nn.relu(what1),self.gamma1,self.beta1) 
    what2 = tf.matmul(hhat1, self.W2) + self.b2   
    hhat2 = self.normalisation(tf.nn.relu(what2),self.gamma2,self.beta2)
    # Compute output
    output = tf.matmul(hhat2, self.W3) + self.b3
    return tf.nn.softmax(output)

  def var(self,y_pred):
    """
    Calculate variance 
    """
    y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
    std_dev = np.std(y_pred_tf) #Calculates standard deviation
    variance = (std_dev**2) # calculate variance
    return variance


## Train Model

In [ ]:
# Set number of epochs
NUM_EPOCHS = 10

In [ ]:
# Initialize model using GPU
mlp_on_gpu = MLP(X_train.shape[1], 256,128, label_size, device='gpu')

train_loss = []
val_loss=[]
seed_ = []
train_accuracy_ = []
train_var_ =[]
val_accuracy_ = []
val_var_ =[]
time_start = time.time()

for epoch in range(NUM_EPOCHS):
  val_loss_total = tf.zeros([1,1], dtype=tf.float32)
  loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)

  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100, seed=epoch*(seeds)).batch(100)
  validate_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(100, seed=epoch*(seeds)).batch(100)

  #creating y_train and y_val after each shuffled data above
  for inputs, outputs in train_ds:
    preds = mlp_on_gpu.forward(inputs)
    loss_total_gpu = loss_total_gpu + mlp_on_gpu.loss(preds, outputs)
    mlp_on_gpu.backward(inputs, outputs)
    a=tf.equal(tf.round(preds,1), tf.round(outputs,1))
    b=tf.reduce_mean(tf.cast(a, "float"))
    train_accuracy_.append(b)

  train_accuracy =tf.reduce_mean(tf.cast(train_accuracy_, "float"))

  for inputs, outputs in validate_ds:
    preds = mlp_on_gpu.forward(inputs)
    val_loss_total = val_loss_total + mlp_on_gpu.loss(preds, outputs)
    a=tf.equal(tf.round(preds,1), tf.round(outputs,1))
    b=tf.reduce_mean(tf.cast(a, "float"))
    val_accuracy_.append(b)

  validation_accuracy =tf.reduce_mean(tf.cast(val_accuracy_, "float"))

  print('Number of Epoch = {} - Average train CCE:= {}'.format(epoch + 1, np.sum(loss_total_gpu) / X_train.shape[0]))
  print('Number of Epoch = {} - Average val CCE:= {}'.format(epoch + 1, np.sum(val_loss_total) / x_val.shape[0]))
  print()
  print('Train Accuracy = {}'.format(train_accuracy*100))
  print('Val Accuracy = {}'.format(validation_accuracy*100))
  print()
  
  print("================================================================")

  train_loss.append(float(loss_total_gpu))
  #val_loss.append(float(val_loss_total))
  seed_.append((epoch+1)*(2785))
  
time_taken = time.time() - time_start
print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))

## One Step Inference

In [ ]:
test_loss_total =tf.zeros([1,1], dtype=tf.float32)

for inputs, outputs in test_ds:
  preds = mlp_on_gpu.forward(inputs)
  test_loss_total = test_loss_total + mlp_on_gpu.loss(preds, outputs)
  a=tf.equal(tf.round(preds,1), tf.round(outputs,1))
  b=tf.reduce_mean(tf.cast(a, "float"))
  val_accuracy_.append(b)
  
test_accuracy =tf.reduce_mean(tf.cast(val_accuracy_, "float"))
print('Test Accuracy = {}'.format(test_accuracy*100))
print('Test Average CCE: {:.4f}'.format(np.sum(test_loss_total.numpy()) / X_test.shape[0]))
#print('Test Variance = {}'.format(test_variance*100))

Test Accuracy = 91.76372528076172
Test Average CCE: 0.0073
